In [55]:
import numpy as np

# **Implementation From Scratch**

In [56]:
class activation_functions:

  @staticmethod
  def step(x : int):
    return 0 if x < 0 else 1

  @staticmethod
  def sigmoid(x : int):
    return 1 / (1 + np.e ** -x)

  @staticmethod
  def tanh(z : int):
    return (np.e ** z - np.e ** -z) / (np.e ** z + np.e ** -z)

  @staticmethod
  def relu(x : int):
    return max(0, x)

  @staticmethod
  def leaky_relu(x : int):
    return max(0.01 * x, x)

  @staticmethod
  def softmax(z : np.array):
    z = np.array(z)
    return np.divide(np.e ** z, np.sum(np.e ** z))

In [57]:
class loss_functions:

  @staticmethod
  def MAE(y_pred : np.array, y_true : np.array):
    return np.abs(np.mean(np.subtract(y_true, y_pred)))

  @staticmethod
  def MSE(y_pred : np.array, y_true : np.array):
    return np.abs(np.mean(np.subtract(y_true, y_pred) ** 2))

  @staticmethod
  def CrossEntropyLoss(y_pred : np.array, y_true : np.array):
    y_pred = activation_functions.softmax(y_pred)
    loss = 0
    for i in range(len(y_true)):
      loss += -1 * y_true[i] * np.log(y_pred[i])
    return loss

In [58]:
x = np.array(
    [
        [1.2, 0.7],
        [0.3, 0.5],
        [1.0, 1.5]
    ]
)

y = np.array(
    [0, 1, 0]
)

In [59]:
np.random.seed(71)

In [60]:
class SLP: # Single Layer Perceptron (A Neuron)

  def gradient_descent(self, x : np.array, y : np.array, epochs : int, lr : float, print_after_epoch : int):
    num_features = x.shape[1]
    weights = np.array(np.random.randn(num_features))
    bias = np.random.randn()
    n = len(x)

    grad_weights = np.zeros(num_features)

    for epoch in range(epochs):
      y_pred = activation_functions.sigmoid(np.dot(x, weights) + bias)

      loss = loss_functions.MSE(y_pred, y)

      grad_weights = (1 / n) * np.dot(np.transpose(x), np.subtract(y_pred, y))
      grad_bias = np.mean(np.subtract(y_pred, y))

      weights -= np.dot(lr, grad_weights)
      bias -= lr * grad_bias

      if (epoch + 1) % print_after_epoch == 0:
        print(f"Epoch : {epoch + 1} | Loss : {loss} | Weights : {weights} | Bias : {bias}")

    return weights, bias

  def fit(self, x : np.array, y : np.array, epochs : int, lr : float, print_after_epoch : int):
    self.weights, self.bias = self.gradient_descent(x, y, epochs, lr, print_after_epoch)

  def predict(self, x : np.array):
    return activation_functions.sigmoid(np.dot(x, self.weights) + self.bias)

In [61]:
Model = SLP()
Model.fit(x, y, 10000, 0.01, 1000)

Epoch : 1000 | Loss : 0.12184642328645436 | Weights : [-0.96272761 -1.14227645] | Bias : 0.6428471501332358
Epoch : 2000 | Loss : 0.08290881740586105 | Weights : [-1.59956365 -1.29490762] | Bias : 1.3081630365594756
Epoch : 3000 | Loss : 0.05788362758537375 | Weights : [-2.13953251 -1.42944392] | Bias : 1.8481250782700485
Epoch : 4000 | Loss : 0.04167792754677702 | Weights : [-2.59859797 -1.54478543] | Bias : 2.2979188951390204
Epoch : 5000 | Loss : 0.030953710804792308 | Weights : [-2.99385699 -1.6444761 ] | Bias : 2.679675867837722
Epoch : 6000 | Loss : 0.02364916009057605 | Weights : [-3.33865582 -1.73159783] | Bias : 3.009187185150956
Epoch : 7000 | Loss : 0.018523985806067775 | Weights : [-3.64309873 -1.80858333] | Bias : 3.2977997327909483
Epoch : 8000 | Loss : 0.014826246836547414 | Weights : [-3.9148219  -1.87731191] | Bias : 3.5537833020187692
Epoch : 9000 | Loss : 0.012090142996969172 | Weights : [-4.1596449  -1.93923395] | Bias : 3.7832749739850575
Epoch : 10000 | Loss : 0.0

In [62]:
round(Model.predict([1.2, 0.7]))

0

# **Implementation Using PyTorch**

In [63]:
import torch
import torch.nn as nn

In [64]:
Model = nn.Sequential(
  nn.Linear(2, 1, bias = True),
  nn.Sigmoid()
)

In [65]:
optimizer = torch.optim.Adam(Model.parameters(), lr = 0.01)
loss_func = nn.MSELoss()

In [66]:
epochs = 10000

for _ in range(epochs):
  y_pred = Model(torch.FloatTensor(x))
  loss = loss_func(y_pred, torch.FloatTensor(y))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [67]:
model_state_dict = Model.state_dict()

In [68]:
weights = model_state_dict['0.weight'].numpy()
bias = model_state_dict['0.bias'].numpy()

print("Weights:", weights)
print("Bias:", bias)

Weights: [[-0.00042698 -0.00040582]]
Bias: [-0.6935576]


In [69]:
with torch.no_grad():
  print(round(float(Model(torch.FloatTensor([1.2, 0.7])))))

0
